В данном файле я преобразую таблицу по источнику категоризации Virustotal, чтобы использовать в дальнейшем эту таблицу для сравнения с другими источниками  

1. Чтение и обработка файлов: Читаются все CSV файлы из директории 'api categorization_v1', содержащие столбцы 'url' и 'cats', где категории разделяются на отдельные столбцы.

2. Объединение данных: Все данные из файлов объединяются в общий DataFrame combined_df, после чего создается новый столбец 'combined_cats', объединяющий все категории для каждого URL в одну строку.

3. Преобразование и нормализация: Столбец 'combined_cats' преобразуется в словарь, а затем нормализуется, создавая отдельные столбцы для каждой категории.

4. Сохранение результата: Итоговый DataFrame сохраняется в новый CSV файл 'новая категоризация аpi virustotal.csv'.

1. Загрузка библиотек 

In [1]:
import re
import os
import pandas as pd
import ast

2.  Чтение и обработка файлов:

In [2]:

# Путь к директории с файлами
input_directory = 'api categorization_v1'

# Создаем пустой DataFrame для объединения данных
combined_df = pd.DataFrame()

# Проходим по всем файлам в директории
for filename in os.listdir(input_directory):
    if filename.startswith('category_result_month_new') and filename.endswith('.csv'):
        file_path = os.path.join(input_directory, filename)
        
        # Читаем файл CSV
        df = pd.read_csv(file_path)
        
        # Предполагаем, что в каждом файле есть колонки 'url' и 'cats'
        if 'url' in df.columns and 'cats' in df.columns:
            # Разделяем категории, если их несколько, и создаем отдельные колонки для каждой
            df_expanded = df['cats'].str.split(',', expand=True)
            df_expanded.columns = [f'cats{i+1}' for i in range(df_expanded.shape[1])]
            
            # Объединяем с колонкой 'url'
            df = pd.concat([df[['url']], df_expanded], axis=1)
            
            # Добавляем данные в общий DataFrame
            combined_df = pd.concat([combined_df, df], ignore_index=True)

# Сохраняем объединенный DataFrame в новый CSV файл
#combined_df.to_csv('новая категоризация аpi virustotal.csv', index=False)
print('Объединение файлов завершено, результат сохранен в "объединенная_таблица.csv"')


Объединение файлов завершено, результат сохранен в "объединенная_таблица.csv"


3.  Преобразование и нормализация:

Этот код объединяет категории, указанные в столбцах 'cats1' до 'cats7', в один столбец 'combined_cats' для каждого URL, исключая пустые значения. Затем он удаляет исходные столбцы с категориями, оставляя только столбцы 'url' и 'combined_cats'.

In [4]:
# Список столбцов с категориями
cats_columns = ['cats1', 'cats2', 'cats3', 'cats4', 'cats5', 'cats6', 'cats7']

# Создание нового столбца, объединяющего категории в одну строку
combined_df['combined_cats'] = combined_df[cats_columns].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)

# Удаление оригинальных столбцов с категориями
df_combined = combined_df[['url', 'combined_cats']]

Этот код преобразует значения в столбце 'combined_cats' в словарь для каждой строки, исправляя форматирование ключей, если это необходимо. Затем данные нормализуются в отдельные столбцы, где источники становятся колонками, а категории — значениями. Наконец, столбец 'url' добавляется обратно в новый DataFrame для создания окончательной структуры данных.

In [5]:
def safe_literal_eval(value):
    try:
        # Заменяем некорректно отформатированные элементы, если необходимо
        formatted_value = re.sub(r"([a-zA-Z0-9_]+):", r'"\1":', value)  # Убедитесь, что ключи правильно заключены в кавычки
        return ast.literal_eval(formatted_value)
    except (ValueError, SyntaxError):
        return {}

# Преобразуем значения столбца combined_cats в словарь для каждой строки
df_combined['combined_cats_dict'] = df_combined['combined_cats'].apply(safe_literal_eval)

# Нормализуем столбец 'combined_cats_dict' для создания DataFrame с источниками как столбцами и категориями как значениями
df_expanded = pd.json_normalize(df_combined['combined_cats_dict'])

# Добавляем обратно столбец 'url' в новый DataFrame
df_expanded = pd.concat([df_combined['url'], df_expanded], axis=1)


C:\Users\rozhk\AppData\Local\Temp\ipykernel_5688\3527323785.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['combined_cats_dict'] = df_combined['combined_cats'].apply(safe_literal_eval)


4. Сохранение результата: 

In [7]:
df_expanded.to_csv('новая категоризация аpi virustotal.csv', index=False)